### **MinION sequencing**

The purpose of DNA sequencing is to obtain the nucleotide arrangement of our genetic material. The development of sequencing technologies in the last years has allowed the identification of characteristic genes of each species and the resolution of the genetic basis of diseases. Technological advances have led to the development of massive or new generation sequencing methods (NGS, Next Generation Sequencing) where millions of independent sequencing processes can be done simultaneously, very quickly and at low cost.

Third generation sequencing allows to obtain longer sequences and do the analysis just with one DNA molecule. In this context, Oxford Nanopore Technologies (ONT) has designed the MinION device, in which the sequencing is carried out by the use of nanopores inserted in a synthetic membrane. The passage of each base of a single-stranded sequence by the nanopore translates into a change in the potential difference, recording a characteristic electrical signal for each base and its sequence of the environment.

![alt text](https://nanoporetech.com/sites/default/files/s3/sequencing-animated.gif)

The generated electric signal, also called events, are placed in a FAST5 format file. . In this way, the first step to be carried out after the sequencing is the naming of bases or also known as Basecalling. This is the process which translates the raw electrical signal from an ONT sequencer to a DNA sequence.

For this step we will use Albacore software. Albacore runs from the command line with some arguments as the directory with the FAST5 files, the output directory, the number of computing threads that will be used and information about the kit employed to build the library.



In [ ]:
!read_fast5_basecaller.py -i data/Agalactiae/Data_MinION/raw_1D/pass -r -t 48 -s data/Agalactiae/Outputs/Albacore -o fastq,fast5 -c r94_450bps_linear.cfg


After using this application, we obtain a file in FASTQ format, which is a file that can be processed using various libraries and software packages. This kind of file is a text file with information about the nucleotid sequence and the quality score of each nucleotid in the sequence. A FASTQ file is characterized by four lines per sequence.

- Line 1 begins with an '@' character and is followed by a sequence identifier and an optional description (such as a FASTA title line).
@SEQ_ID
- Line 2 to represent the nucleotide sequence using one-letter codes.
GATTTGGGGTTCAAAGCAGTATCGATCAAATAGTAAATCCATTTGTTCAACTCACAGTTT
- Line 3 starts with a '+' character and it is optionally followed by the same identifier as the sequence again (and any other description).
+
- Line 4 encodes the quality values for the sequence in line 2, and must contain the same number of symbols as the letters in the sequence.
! '' * ((((* +)) %%% ++) (%%%%). 1 * - + * '')) ** 55CCF >>>>>> CCCCCCC65

```
@SEQ_ID
GATTTGGGGTTCAAAGCAGTATCGATCAAATAGTAAATCCATTTGTTCAACTCACAGTTT
+
! '' * ((((* +)) %%% ++) (%%%%). 1 * - + * '')) ** 55CCF >>>>>> CCCCCCC65
```

From this file, a FASTA file can be obtained, which also has information about the nucleotid sequence.  

- Line 1 starts with a '>' character  and it is followed by a sequence identifier and an optional description.
SEQ_ID
- Line 2 is the letter-encoded sequence
GATTTGGGGTTCAAAGCAGTATCGATCAAATAGTAAATCCATTTGTTCAACTCACAGTTT

```
>SEQ_ID
GATTTGGGGTTCAAAGCAGTATCGATCAAATAGTAAATCCATTTGTTCAACTCACAGTTT
```

Once the basecalling is done, an analysis of the quality, number, length and GC content will be made. For this, we will use the Python programming language and the Biopython module, as well as loading both data processing and plotting libraries.
 
The following lines import the Biopython module and convert the FASTQ file in FASTA:


In [ ]:
#Biopython module imports for loading and transforming sequence files
from Bio import SeqIO

SeqIO.convert("data/agalactiae/merged-output.fastq", "fastq", "data/agalactiae/merged-output.fasta", "fasta")

With the FASTA file, we can create a dataframe with the information about the sequence. With this dataframe, it will be possible to have summary tables, plots and it is also possible to perform filters. The following defines a function that takes a FASTA/FASTQ file and builds a dataframe with all the information structured as columns.

In [ ]:
from Bio.SeqUtils import GC

#Numpy and pandas to manage our data
import numpy as np
import pandas as pd

#We will use reagular expressions for extract the read id
import re

#Function to build a pandas dataframe from a FASTA or FASTQ file
def read_sequence_file(filename, format):
    rows = []
    columns = ['id', 'sequence', 'read_length', 'gc_content']
    if(format == 'fastq'):
        columns.append('avg_quality')
    #Building each row with the read and its features
    for seq_record in SeqIO.parse(filename, format):
        if(format == 'fastq'):
            quality_score = np.mean(seq_record.letter_annotations["phred_quality"])
        read_id = seq_record.description
        if(format == 'fastq'):
            rows.append([read_id, seq_record.seq, len(seq_record.seq), GC(seq_record.seq), quality_score])
        else:
            rows.append([read_id, seq_record.seq, len(seq_record.seq), GC(seq_record.seq)])  
        
    df = pd.DataFrame(rows, columns=columns)
    return df



# ----- Place the path to your reads file here ----- #
seq_df = read_sequence_file('data/metagenomics/metagenomics_exercise_sample.fa', 'fasta')

The following line of code will show a summary table with following data about our dataframe:

- The number of sequences contained in the FASTA/FASTQ file or what is the same, the number of reads sequenced by the device.
- The standard deviation of each parameter.
- Minimum and maximum of each parameter.

In [ ]:
seq_df.describe()

**Visualizations**

In the following steps, we will build plots to visualize the data that we want to analyze:

***Read length***

The following plot shows the read length distribution. Length range distribution is in the x axis and the read count in the y axis.

In [ ]:
#Using matplotlib to generate GC content histogram
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import matplotlib

#Setting the plot size
matplotlib.rcParams['figure.figsize'] = (9, 8)

#We can customize the plot using the pre-defined style called “ggplot” to emulate ggplot (popular R vizualization package) style 
plt.style.use('seaborn-notebook')

#On the plt.hist parameters we can set the number of bins and the bar color using labels as 'blue', 'red', 'yellow'... or using a RGB code.
plt.hist(seq_df['read_length'], bins=50, color='#ef8a62')
plt.title('Read length histogram')
plt.xlabel('Read length in bp')
plt.ylabel('Count')

plt.show()

***Guanine-Cytosine content (GC)***

GC content is the percentage of guanine or cytosine nitrogenous bases present in a DNA or RNA molecule. The Guanine - Cytosine pair is bound by three hydrogen bonds, which increases the stability of the DNA molecule. In this way, DNA with low GC content is less stable than DNA with high GC content, hence it is useful to analyze the GC content of what we are analyzing, as for example it has been observed that bacteria with a High GC content in their DNA is more easily autolysed, thus reducing the longevity of the cell per se. The GC content is variable among the different organisms, which seems to indicate that there is a contribution to the variation in selection, mutational bias and DNA repair associated with partial recombination.
 
Due to the nature of the genetic code, it is practically impossible for an organism to have a genome with a GC content approaching 0% or 100%. In this way, checking the percentage of GC content of our readings can help identifying possible artifacts or errors in our data that we must discard so as not to influence our final result.
 
The distribution of GC content is shown in the following plot. The percentage in GC and in the x axis and the number of reads are represented in the y axis.

In [ ]:
#Building histogram with the gc content column of the dataframe
plt.hist(seq_df['gc_content'], bins=50, color='#ef8a62')
plt.title('GC content histogram')
plt.xlabel('GC')
plt.ylabel('Count')
plt.show()

***Quality***

 
The following graph shows the quality distribution. The average quality value assigned to each sequence is represented on the x axis and the read count on the y axis.

In [ ]:
#Only works with FASTQ files
plt.hist(seq_df['avg_quality'], bins=50, color='#ef8a62')
plt.title('Average quality histogram')
plt.xlabel('Average quality of the read')
plt.ylabel('Count')

plt.show()

Once the quality control and filtering is performed, we proceed with the classification of sequences, that is, the identification of the different organisms (plants, animals, bacteria, etc.) that we have sequenced.
 
For this, we must follow the next steps:

1.- Create a database from our references file to be used by BLAST.  
2.- Run BLAST to classify the reads in our sample.  
3.- Use Python for filtering and getting the key information of the BLAST output.  
4.- Plot the results to see a representation of the proportion of each species in the initial sample.  

In this case, we will use BLAST software: 

[BLAST](https://blast.ncbi.nlm.nih.gov) (stands for Basic Local Alignment Search Tool) is a tool used for comparing aminoacid sequences of proteins and nucleotids of DNA sequences. 
 
BLAST needs a database where the reference sequences of the genomes are indexed for comparison. In this case we have a sample of 210 reads and there are 7 target genomes. These references are placed in one FASTA file. BLAST has different commands depending on what we want to do.

First, we will create our database with our references file (all_metagenomics_references.fasta). The command makeblastdb will have the following arguments:


In [ ]:
!makeblastdb -in data/metagenomics/references/database/metagenomics_references_withnames.fasta -parse_seqids -dbtype nucl

Once our database is created, we are ready to query it. The input is a FASTA file with 210 unidentified sequences. The query command has the following arguments:

In [ ]:
!blastn -query data/metagenomics/metagenomics_exercise_sample.fa -db /home/jovyan/notebooks/data/metagenomics/references/metagenomics_references_withnames.fasta -task blastn -dust no -outfmt "10 qseqid positive sseqid" -max_hsps 1 -max_target_seqs 1 -num_threads 2 > blast_metagenomics.csv

We get the output on csv format as it is easier to be processed by the pandas package to analyze and plot the results.

Data must be filtered in order to know which sequences aligned with any reference and which sequences did not. The data is grouped by subject so we have the count of sequences that aligned with each reference.

Finally, the number of sequences aligned to a reference is transformed into the proportion of reads that are aligned to that reference.



In [ ]:
import numpy as np
import pandas as pd

df = pd.read_csv('blast_metagenomics.csv', names=['id', 'score', 'subject'], header=None)

#Colocamos un filtro de matches. Si tiene un número de matches < 100, se considera que no alinea.
aligned_sequences = df.loc[df['score'] > 100, :]
unaligned_reads_count = len(df) - len(aligned_sequences)

#Agrupamos por especie y generamos tabla de especie -> # de seqs que alinean
counts_df = aligned_sequences.groupby(['subject']).size().reset_index(name='counts')
counts_df = counts_df.append({'subject':'unaligned', 'counts':unaligned_reads_count}, ignore_index=True)

#Transformamos num de seqs en proporción de seqs
#counts_df['counts'] = counts_df['counts'].map(lambda x: (x/(210 - unaligned_reads_count)) * 100)
counts_df.columns = ['subject', 'proportion']

display(counts_df)

The following code takes the species proportions and builds a plot using the matplotlib module.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (15,15)

colors = ['#F15854', '#5DA5DA', '#FAA43A', '#60BD68', '#F17CB0', '#B276B2', '#DECF3F']
labels = ['Alligator mississippiensis', 'Gallus gallus', 'Bos taurus', 'Equus caballus', 'Oreochromis niloticus', 'Rattus norvegicus', 'Ovis aries']
explode = (0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02)

plt.title('% of each species on the sample')
plt.pie(np.asarray(counts_df['proportion'].iloc[0:7]), autopct='%.2f', colors=colors, labels=labels, explode=explode, shadow=None, startangle=90)
centre_circle = plt.Circle((0,0),0.75,color='white', fc='white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)
plt.show()